# Programming and Mathematics for AI
## Coursework

This notebook contains all solutions annotated via markdown for the Programming and Mathematics for AI coursework.

In [1]:
# Imports for the whole project
import math
import numpy as np
import pandas as pd
import matplotlib as mp
from collections import defaultdict

## Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0 and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible.

You can implement one of the two (or both, for no extra point) game modes:
- The time spent on a cell is the number on this cell
- The time spent on a cell is the absolute of the difference between the previous cell the agent was on and the current cell it is on

In order to solve this problem, you will provide 3 algorithms:
- A first baseline of your choosing. E.g. it can be any search algorithm, or an algorithm using heuristics. It doesn’t have to perform fast or well, but should be better than random movements.
- Dijkstra's algorithm
- Ant colony optimization algorithm

You should describe the algorithms and compare them. Are they always solving the problem? How long do they take depending on the size of the maze?

## Creating the Game Board

I have separated some functions for the creation of the game board as a numpy array, as well as the getNeighbours function as it can be used in both Dijkstra's Algorithm and in the A* pathfinding algorithm (as they are very similar). If you were to work under the assumption that the h() function were always equal to 0, the two algorithms would function the same.

In [747]:
def defineGrid(width, height):
    return np.random.randint(10, size=(width, height))

# Defined here as returning all neighbours can be used
# with Dijkstra's Algorithm and with A* Pathfinding
def getNeighbours(current, width, height):
    neighbours = [
        (current[0], current[1] + 1),
        (current[0], current[1] - 1),
        (current[0] + 1, current[1]),
        (current[0] - 1, current[1])
    ]
    neighbours = [x for x in neighbours if (-1 not in x and x[0] < height and x[1] < width)]
    return neighbours

## Baseline

In this baseline approach, the algorithm is only choosing values either to it's right or beneath it in order to progress towards the end point in the bottom right of the board. It will detect if it is at the edge of the board on either side at which point it will only move in the direction available. Slightly better than random movements as there is no likelihood for backtracking, however still not as efficient as possible as it cannot move up and around larger numbers in order to avoid high influences on the overall time as well as it does not consider the overall best route and will just take the smallest number between the right and bottom movements, meaning it could make a move right to increase time by 1, but then be forced by the edge to move down through many large numbers, not necessarily very efficient. As such I believe it to be a suitable baseline for this task.

In [748]:
def newPos(grid, x, y, width, height):
    if(x + 1 > width - 1):
        return (y + 1, x)
    elif(y + 1 > height - 1):
        return (y, x + 1)
    elif(grid[y][x + 1] >= grid[y + 1][x]):
        return (y + 1, x)
    else:
        return (y, x + 1)

def runBaseline(grid, width, height):
    pos = (0, 0)
    steps = []
    steps.append((pos, grid[pos[0], pos[1]]))
    while((pos[0] + 1, pos[1] + 1) != grid.shape):
        pos = newPos(grid, pos[1], pos[0], width, height)
        steps.append(((pos[0], pos[1]), grid[pos[0], pos[1]]))
    return steps, sum([grid[x[0], x[1]] for x in steps])

## Dijkstra's Algorithm

Dijkstra's algorithm implemented below allows for movement up and down as well as back tracking through previous values to adjust and to make sure it is taking the optimal route by testing the distance from the starting point through to the ending point along a number of routes. In a sense it is somewhat of a brute force method and so is generally slower than more optimal algorithms, however still efficient at finding the correct route! It will alter these values in a dictionary of distances that allow it to optimise over time and find the shortest possible route, however this is almost a brute force method as it checks every possible node for it's distance from the starting point to the end point. Not as fast as something like the algorithm above but much more accurate.

In [749]:
def generateNodes(width, height):
    nodes = []
    for i in range(0, height):
        for j in range(0, width):
            nodes.append((i, j))
    return nodes

def generateDistances(nodes):
    distances = {}
    for node in nodes:
        if node == (0, 0):
            distances[node] = 0
        else:
            distances[node] = math.inf
    return distances

def generatePrevious(nodes):
    previous = {}
    for node in nodes:
        previous[node] = None
    return previous

def getSmallestDist(nodes, dist):
    low = math.inf
    for node in nodes:
        if(dist[node] < low):
            low = dist[node]
            pos = node
    return pos

def backtrack(current, prev):
    path = []
    total = 0
    while current != None:
        path.append((current, grid[current[0], current[1]]))
        total += grid[current[0], current[1]]
        current = prev[current]
    return path[::-1], total

def runDijkstra(grid, width, height):
    nodes = generateNodes(width, height)
    dist = generateDistances(nodes)
    prev = generatePrevious(nodes)
    while nodes:
        current = getSmallestDist(nodes, dist)
        nodes.remove(current)
        for neighbour in getNeighbours(current, width, height):
            alt = dist[current] + grid[neighbour[0], neighbour[1]]
            if alt < dist[neighbour]:
                dist[neighbour] = alt
                prev[neighbour] = current
    return backtrack(current, prev)

## A* Pathfinding Algorithm

In [750]:
def getDistance(node1, node2):
    posVector = tuple(map(lambda pos1, pos2: abs(pos1 - pos2), node1, node2))
    return math.pow(posVector[0], 2) + math.pow(posVector[1], 2)

def g(startNode, currentNode, grid):
    return getDistance(startNode, currentNode)

def h(currentNode, endNode):
    return getDistance(currentNode, endNode)

def f(G, H):
    return G + H

def reconstructPath(cameFrom, current):
    path = []
    total = 0
    while current in cameFrom.keys():
        path.append((current, grid[current[0], current[1]]))
        total += grid[current[0], current[1]]
        current = cameFrom[current]
    path.append(((0, 0), grid[0, 0]))
    total += grid[0, 0]
    return path[::-1], total

def getSmallestFScore(op, fScore):
    low = math.inf
    pos = None
    for node in op:
        if(fScore[node] < low):
            low = fScore[node]
            pos = node
    return pos
    
def runAStarPathfinding(width, height, grid, weight):
    start = (0, 0)
    end = (height - 1, width - 1)
    
    op = [start]
    cameFrom = {}

    gScore = {}
    gScore = defaultdict(lambda: math.inf, gScore)
    gScore[start] = 0

    fScore = {}
    fScore = defaultdict(lambda: math.ind, fScore)
    fScore[start] = h(start, end)
    
    while op:
        current = getSmallestFScore(op, fScore)
        if(current == end):
            return reconstructPath(cameFrom, current)
        op.remove(current)
        for child in getNeighbours(current, width, height):
            tentGScore = gScore[current] + ((1 / weight) * grid[child[0], child[1]])
            if(tentGScore < gScore[child]):
                cameFrom[child] = current
                gScore[child] = tentGScore
                fScore[child] = tentGScore + h(child, end)
                if(child not in op):
                    op.append(child)
    return Exception("Failure")

## Testing of Accuracy

In [758]:
maxWeight = 20
iterations = 10

validity = []

for weight in range(1, maxWeight + 1):
    validity.append([])
    for i in range(0, iterations):
        width = 10
        height = 10
        grid = defineGrid(width, height)
        baseline = runBaseline(grid, width, height)
        dijkstra = runDijkstra(grid, width, height)
        aStar = runAStarPathfinding(width, height, grid, 1 / weight)
        validity[weight - 1].append(aStar == dijkstra)

data = np.array(validity, dtype=np.bool)
for i, group in enumerate(data):
    weight = i + 1
    print("Weight: %i, True: %i" % (weight, np.sum(group)))

Weight: 1, True: 0
Weight: 2, True: 1
Weight: 3, True: 5
Weight: 4, True: 4
Weight: 5, True: 2
Weight: 6, True: 2
Weight: 7, True: 2
Weight: 8, True: 3
Weight: 9, True: 2
Weight: 10, True: 3
Weight: 11, True: 3
Weight: 12, True: 2
Weight: 13, True: 1
Weight: 14, True: 4
Weight: 15, True: 3
Weight: 16, True: 3
Weight: 17, True: 2
Weight: 18, True: 4
Weight: 19, True: 2
Weight: 20, True: 4
